In [2]:
import sqlite3
import pandas as pd
import numpy

In [3]:

def write_sentences_in_sqlite(input_, output_, file_num):
    print("write_sentences_in_sqlite function is running")

    with open(input_, "r", encoding="charmap") as d:
        if str(type(input_)) == "<class 'list'>":
            sentences = input_
        else:
            sentences = d.readlines()

        emails = []
        data = []
        email_1 = ""
        for sentence in sentences:
            if sentence.split(",") != ['\n']:
                try:
                    if sentence == sentences[0]:
                        continue
                    email_1 = sentence.strip().split(",")[7].replace('"', "").replace("From:", "").replace('"', '').strip().lower()

                    if ";" in email_1:
                        email_1 = email_1.split(';')[0].strip()
                    email_2 = sentence.split(",")[8].replace('"', "").replace("To:", "").replace('"', '').strip().lower()
                    if ";" in email_2:
                        email_2 = email_2.split(';')[0].strip()
                    if email_1 not in emails:
                        emails.append(email_1)
                    if email_2 not in emails:
                        emails.append(email_2)

                    data.append([
                        sentence.split(",")[0].replace('"', ""),
                        sentence.split(",")[6].replace('"', "").replace("DateTime:", ""),
                        "EMID"+ str(sentence.split(",")[0].replace('"', "")),
                        emails.index(email_1),
                        emails.index(email_2),
                        sentence.split(",")[9].replace("Sentence:", "").strip(),
                        sentence.split(",")[10].replace("PROPN:", ""),
                        sentence.split(",")[11].replace("VERB:", ""),
                        sentence.split(",")[12].replace("ADJ:", ""),
                        sentence.split(",")[13].replace("NOUN:", ""),
                        numpy.round(float(sentence.split(",")[1]), 4),
                        numpy.round(float(sentence.split(",")[2]), 4),
                        sentence.split(",")[3],
                        numpy.round(float(sentence.split(",")[4]), 4)
                    ])
                except:
                    raise Exception

        df = pd.DataFrame(data, columns=[
            "ID", "date", "email_id", "from(email index)",
            "to(email index)", "sentence", "propn", "verb",
            "adjective", "noun", "jaccard_similarity",
            "cosine_similarity", "levenshtein_distance",
            "euclidean_distance"
        ])

        conn = sqlite3.connect(output_)
        df.to_sql("tbl_sentence", conn, if_exists="replace", index=False)
        conn.close()
        ID = 1
        number_of_emails = 0

        # Connect to the SQLite database
        conn = sqlite3.connect(output_)
        cursor = conn.cursor()

        # Create the table if it doesn't exist
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS emails (
                ID INTEGER,
                email TEXT
            )
        ''')

        # Insert the emails into the table
        for email in emails:
            try:
                if email != emails[0]:
                    if number_of_emails % 100 == 0:
                        print(f"{number_of_emails} email addresses are recorded")
                    number_of_emails += 1
                    cursor.execute("INSERT INTO emails (ID, email) VALUES (?, ?)", (ID, email))
                    ID += 1
            except:
                pass

        # Commit the changes and close the connection
        conn.commit()
        conn.close()
        sentence_list = []
        email_ids = []
        ID = 0

        # Connect to the database
        conn = sqlite3.connect(output_)
        cursor = conn.cursor()

        # Create the email table if it doesn't exist
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS tbl_email (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            email_id TEXT NOT NULL,
            email_date TEXT NOT NULL,
            email_time TEXT NOT NULL,
            email_subject TEXT,
            from_id INTEGER NOT NULL,
            to_id INTEGER NOT NULL,
            body TEXT NOT NULL
        )
        """)
        
        # Iterate over the sentences
        for sentence in sentences:
            sentence_list = []
            email_id = sentence.split(",")[0].replace('"', "")
            if email_id not in email_ids:
                email_ids.append(email_id)
            try:    
                for s in sentences:
                    s_email_id = s.split(",")[0].replace('"', "")
                    if s_email_id == email_id and s.split(",")[9].replace("Sentence:", "").strip() not in sentence_list:
                        sentence_list.append(s.split(",")[9].replace("Sentence:", "").strip())
                email_1 = sentence.split(",")[7].replace('"', "").replace("From:", "").replace('"', '').strip().lower()
                email_1 = email_1.split(';')[0].strip() if ";" in email_1 else email_1
                email_2 = sentence.split(",")[8].replace('"', "").replace("To:", "").replace('"', '').strip().lower()
                email_2 = email_2.split(';')[0].strip() if ";" in email_2 else email_2
                subject = sentence.split(",")[14] if len(sentence.split(",")) > 14 else ''
                body = str(sentence_list).replace("]", "").replace("[", "")
                email_date = sentence.split(",")[6].replace('"', "").replace("DateTime:", "").split()[0]
                email_time = sentence.split(",")[6].replace('"', "").replace("DateTime:", "").split()[1]
                # Insert the email data into the database
                try:
                    cursor.execute("""
                    INSERT INTO tbl_email (email_id, email_date, email_time, email_subject, from_id, to_id, body)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                    """, ("EMID" + email_id, email_date, email_time, subject, emails.index(email_1), emails.index(email_2), body))
                    ID += 1
                except:
                    pass
            except:
                pass
        # Commit the changes to the database and close the connection
        conn.commit()
        conn.close()
write_sentences_in_sqlite("sentence_in_groups.txt", 'tables.db', 0)

write_sentences_in_sqlite function is running
0 email addresses are recorded


In [4]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('tables.db')

# Query the database and store the result in a dataframe
df = pd.read_sql_query("SELECT * from emails", conn)
df1 = pd.read_sql_query("SELECT * from tbl_email", conn)
df2 = pd.read_sql_query("SELECT * from tbl_sentence", conn)



# Close the connection
conn.close()

# Print the dataframe
print(df)

    ID                                              email
0    1                   stephan.schmalhofer@techdata.com
1    2                               noreply@techlinx.com
2    3  /o=exchangelabs/ou=exchange administrative gro...
3    4                       christoph.lindtner@artech.at
4    5                     michaela.altziebler@mavoco.com
5    6                      anruf@benefit-bueroservice.at
6    7                           at_cloud@ingrammicro.com
7    8                             bluyssen@lehmann-it.de
8    9  /o=exchangelabs/ou=exchange administrative gro...
9   10                      stephanie.barth@infinigate.de
10  11                        michaela.liebhart@brz.gv.at
11  12  /o=exchangelabs/ou=exchange administrative gro...
12  13                     fuerst@kardinal-koenig-haus.at
13  14                         partners@lenovopartner.com
14  15  /o=exchangelabs/ou=exchange administrative gro...
15  16  /o=exchangelabs/ou=exchange administrative gro...
16  17        

In [5]:
import pandas as pd
import sqlite3

def communication_streams(input_, output_, file_num):
    conn = sqlite3.connect(output_)
    data = pd.read_sql("SELECT * FROM tbl_email", conn)
    list_of_emails = data.values.tolist()
    sorted_list = []
    com_s = 0
    sorted_communication_stream = []
    for i in list_of_emails:
        try:
            for j in list_of_emails:
                if (i[5] == j[5] or j[5] == j[6]) and (i[6] == j[5] or i[6] == j[6]) and j[
                    1] not in sorted_communication_stream:
                    sorted_list.append((com_s, j))
                    sorted_communication_stream.append(j[1])
            com_s += 1
        except:
            pass

    db = pd.DataFrame({
        "id": [sorted_list.index(sorted_list[0])],
        "comm_stream_id": ["COMM"+str(sorted_list[0][0])],
        "email_id": [sorted_list[0][1][1]]
    })

    for comunication_stream in sorted_list:
        if comunication_stream != sorted_list[0]:
            try:
                new_db = pd.DataFrame({
                    "id": [sorted_list.index(comunication_stream)],
                    "comm_stream_id": ["COMM"+str(comunication_stream[0])],
                    "email_id": [comunication_stream[1][1]],
                })
                db = pd.concat([db, new_db], ignore_index=True, axis=0)
            except:
                pass
    try:
        db.to_sql("communication_streams", conn, if_exists="replace", index=False)
    except:
        pass
communication_streams("", "tables.db",3)

In [6]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('tables.db')

# Query the database and store the result in a dataframe
df = pd.read_sql_query("SELECT * from communication_streams", conn)

# Close the connection
conn.close()

# Print the dataframe
print(df)

    id comm_stream_id  email_id
0    0          COMM0    EMID55
1    1          COMM0   EMID100
2    2          COMM0    EMID42
3    3          COMM0   EMID421
4    4          COMM1  EMID2768
..  ..            ...       ...
67  67        COMM159   EMID722
68  68        COMM175  EMID1637
69  69        COMM176    EMID44
70  70        COMM179    EMID49
71  71        COMM189   EMID480

[72 rows x 3 columns]
